In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
orders_df = spark.read \
.format("csv") \
.option("header", "true") \
.option("inferSchema", "true") \
.load("/public/trendytech/orders_wh/*")

In [3]:
orders_df.show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|       1837|         CLOSED|
|      13|2013-07-25 00:00:...|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:...|       98

In [4]:
orders_df.createOrReplaceTempView("orders")

##### 1.Top 15 customers who placed most number of orders

In [5]:
#dataframe
result1 = orders_df.groupBy("customer_id").count().sort("count",ascending = False).limit(15)
result1.show()

#sparkSQL 
#created tempview as orders - orders_df.createOrReplaceTempView("orders")
result2 = spark.sql("select customer_id, count(order_id) as count from orders group by 1 order by 2 desc limit 15")
result2.show()


+-----------+-----+
|customer_id|count|
+-----------+-----+
|       5897|   16|
|      12431|   16|
|        569|   16|
|       6316|   16|
|      12284|   15|
|       4320|   15|
|       5624|   15|
|       5283|   15|
|        221|   15|
|       5654|   15|
|       6248|   14|
|       3708|   14|
|       1011|   14|
|       8652|   14|
|       4517|   14|
+-----------+-----+

+-----------+-----+
|customer_id|count|
+-----------+-----+
|       5897|   16|
|      12431|   16|
|        569|   16|
|       6316|   16|
|      12284|   15|
|       4320|   15|
|       5624|   15|
|       5283|   15|
|        221|   15|
|       5654|   15|
|       6248|   14|
|       3708|   14|
|       1011|   14|
|       8652|   14|
|       4517|   14|
+-----------+-----+



##### 2. Find the number of orders under each order status

In [6]:
#dataframe
result3 = orders_df.groupBy("order_status").count()
result3.show()

#sparkSQL 
#created tempview as orders - orders_df.createOrReplaceTempView("orders")
result4 = spark.sql("select order_status, count(order_id) as count from orders group by 1")
result4.show()


+---------------+-----+
|   order_status|count|
+---------------+-----+
|PENDING_PAYMENT|15030|
|       COMPLETE|22899|
|        ON_HOLD| 3798|
| PAYMENT_REVIEW|  729|
|     PROCESSING| 8275|
|         CLOSED| 7556|
|SUSPECTED_FRAUD| 1558|
|        PENDING| 7610|
|       CANCELED| 1428|
+---------------+-----+

+---------------+-----+
|   order_status|count|
+---------------+-----+
|PENDING_PAYMENT|15030|
|       COMPLETE|22899|
|        ON_HOLD| 3798|
| PAYMENT_REVIEW|  729|
|     PROCESSING| 8275|
|         CLOSED| 7556|
|SUSPECTED_FRAUD| 1558|
|        PENDING| 7610|
|       CANCELED| 1428|
+---------------+-----+



##### 3.Number of active customers who placed atleast one order

In [7]:
#dataframe

result5 = orders_df.select("customer_id").distinct().count()
print(result5)

#sparkSQL 
#created tempview as orders - orders_df.createOrReplaceTempView("orders")
result6 = spark.sql("select count(distinct customer_id) from orders")
result6.show()


12405
+---------------------------+
|count(DISTINCT customer_id)|
+---------------------------+
|                      12405|
+---------------------------+



##### 4. Customer with most closed orders

In [13]:
#dataframe
result7 = orders_df.where("order_status == 'CLOSED'").groupBy("customer_id").count().sort("count",ascending = False).select("customer_id","count")
print(result7)

#sparkSQL 
#created tempview as orders - orders_df.createOrReplaceTempView("orders")
result8 = spark.sql("select customer_id,count(*) from orders where order_status = 'CLOSED' group by customer_id order by 2 desc")
result8.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|       1833|    6|
|       1363|    5|
|       1687|    5|
|       5493|    5|
|       7948|    4|
|       2768|    4|
|      10263|    4|
|       2236|    4|
|       2403|    4|
|       7879|    4|
|       4573|    4|
|       7850|    4|
|      12431|    4|
|       1521|    4|
|      10111|    4|
|        437|    4|
|      10018|    4|
|       5319|    4|
|       2774|    4|
|       3631|    4|
+-----------+-----+
only showing top 20 rows

+-----------+--------+
|customer_id|count(1)|
+-----------+--------+
|       1833|       6|
|       1363|       5|
|       1687|       5|
|       5493|       5|
|       7948|       4|
|       2768|       4|
|      10263|       4|
|       2236|       4|
|       2403|       4|
|       7879|       4|
|       4573|       4|
|       7850|       4|
|      12431|       4|
|       1521|       4|
|      10111|       4|
|        437|       4|
|      10018|       4|
|       5319|       4|
|       277